# 匯入函式庫

In [19]:
import numpy as np
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings("ignore")

# 匯入資料

In [20]:
data = pd.read_excel('./CAD_AUD_GBP.xlsx', index_col=0, skiprows=3)
data = data.iloc[1:,:]
data.head(15)

,CADUSD Curncy,AUDUSD Curncy,GBPUSD Curncy
1981-01-01 00:00:00,0.8372,1.1814,2.389
1981-01-02 00:00:00,0.837,1.1802,2.382
1981-01-05 00:00:00,0.8413,1.1847,2.4155
1981-01-06 00:00:00,0.8416,1.189,2.4225
1981-01-07 00:00:00,0.8418,1.189,2.4115
1981-01-08 00:00:00,0.8418,1.1865,2.4035
1981-01-09 00:00:00,0.8415,1.1852,2.395
1981-01-12 00:00:00,0.8408,1.18,2.397
1981-01-13 00:00:00,0.8401,1.1793,2.388
1981-01-14 00:00:00,0.8397,1.1808,2.4025


In [21]:
data = data.astype('float64')
data.index = pd.to_datetime(data.index)

In [22]:
data.head()

,CADUSD Curncy,AUDUSD Curncy,GBPUSD Curncy
1981-01-01,0.8372,1.1814,2.3890
1981-01-02,0.8370,1.1802,2.3820
1981-01-05,0.8413,1.1847,2.4155
1981-01-06,0.8416,1.1890,2.4225
1981-01-07,0.8418,1.1890,2.4115


In [23]:
# 轉成月資料
# data = data.resample('M').ffill()

# 資料前處理

In [24]:
log_data = np.log(data)

In [25]:
log_data.head()

,CADUSD Curncy,AUDUSD Curncy,GBPUSD Curncy
1981-01-01,-0.177692,0.166700,0.870875
1981-01-02,-0.177931,0.165684,0.867940
1981-01-05,-0.172807,0.169490,0.881906
1981-01-06,-0.172450,0.173113,0.884800
1981-01-07,-0.172213,0.173113,0.880249


In [26]:
diff_log_data = log_data.diff().dropna()

In [27]:
diff_log_data.head()

,CADUSD Curncy,AUDUSD Curncy,GBPUSD Curncy
1981-01-02,-0.000239,-0.001016,-0.002934
1981-01-05,0.005124,0.003806,0.013966
1981-01-06,0.000357,0.003623,0.002894
1981-01-07,0.000238,0.000000,-0.004551
1981-01-08,0.000000,-0.002105,-0.003323


In [28]:
diff_log_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10435 entries, 1981-01-02 to 2020-12-31
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CADUSD Curncy  10435 non-null  float64
 1   AUDUSD Curncy  10435 non-null  float64
 2   GBPUSD Curncy  10435 non-null  float64
dtypes: float64(3)
memory usage: 326.1 KB


In [29]:
def statistic(forex_name, data):
    if forex_name == 'cad':
        col = 0
    elif forex_name == 'aud':
        col = 1
    elif forex_name == 'gbp':
        col = 2
    else:
        return 'forex_name is not available.'
    
    forex = data.iloc[:,col]
    mean = forex.mean()
    std = forex.std()
    min = forex.min()
    max = forex.max()
    skew = forex.skew()
    kurt = forex.kurtosis()
    ADF, p_value, _1, _2, _3, _4 = adfuller(forex, autolag='BIC')
    
    return mean, std, min, max, skew, kurt, ADF, p_value

In [30]:
statistic('cad', diff_log_data)

(-6.132935332152909e-06,
 0.004435299703995574,
 -0.03251800698699961,
 0.039808113434501136,
 -0.10472665438577913,
 5.0083822592686165,
 -104.74110656061137,
 0.0)

In [31]:
statistic('aud', diff_log_data)

(-4.109673833779984e-05,
 0.007079836469999504,
 -0.10553624714057355,
 0.08276574769364836,
 -0.7151671369350999,
 13.802018388309214,
 -104.15950297840136,
 0.0)

In [32]:
statistic('gbp', diff_log_data)

(-5.349844854833354e-05,
 0.0062319317649937285,
 -0.08395458606779915,
 0.04588529647680195,
 -0.3287690203448818,
 6.141745651216482,
 -98.74616975965519,
 0.0)

# p, d, q selection and linear_prediction, residue computation

In [33]:
p_d_q_select_data = diff_log_data.loc['1981-01-01':'2016-12-31']

In [34]:
p_d_q_select_data

,CADUSD Curncy,AUDUSD Curncy,GBPUSD Curncy
1981-01-02,-0.000239,-0.001016,-0.002934
1981-01-05,0.005124,0.003806,0.013966
1981-01-06,0.000357,0.003623,0.002894
1981-01-07,0.000238,0.000000,-0.004551
1981-01-08,0.000000,-0.002105,-0.003323
...,...,...,...
2016-12-26,0.002164,0.002227,0.000000
2016-12-27,-0.004875,-0.000974,-0.000896
2016-12-28,0.001221,-0.001114,-0.003674
2016-12-29,0.004195,0.005835,0.002859


In [35]:
def grid_search_p_q(data, p_list, d, q_list):
    min_bic = 1000000
    min_bic_order = (0, 0, 0)
    for p in p_list:
        for q in q_list:
            order = (p, d, q)
            model = ARIMA(data, order=order)
            res = model.fit()
            bic = res.bic
            if bic < min_bic:
                min_bic_order = order
                min_bic = bic
                print('min_bic_order:', min_bic_order)
                print('min_bic:', min_bic)
            else:
                print('order:', order, 'doesn\'t make bic smaller')
                print('bic:', bic)
    
    print('min_bic_order:', min_bic_order)
    return min_bic_order

In [36]:
def recursive_true_linear_residue(p_d_q_select_data, order, diff_log_data, currency):
    model = ARIMA(p_d_q_select_data, order=order)
    model_fit = model.fit()
    in_sample_predict = model_fit.predict()
    in_sample_residue = model_fit.resid
    recursive_linear_prediction = in_sample_predict.copy()
    recursive_linear_residue = in_sample_residue.copy()

    true = diff_log_data.copy()[currency]
    for end_index in range(len(p_d_q_select_data), len(true)):
        used_data = true[:end_index]
        model = ARIMA(used_data, order=order)
        model_fit = model.fit()
        recursive_linear_prediction = recursive_linear_prediction.append(model_fit.forecast())
        recursive_linear_residue = recursive_linear_residue.append(true[end_index] - model_fit.forecast())
    
    return true, recursive_linear_prediction, recursive_linear_residue

In [37]:
def rolling_true_linear_residue(p_d_q_select_data, order, diff_log_data, currency):
    model = ARIMA(p_d_q_select_data, order=order)
    model_fit = model.fit()
    in_sample_predict = model_fit.predict()
    in_sample_residue = model_fit.resid
    rolling_linear_prediction = in_sample_predict.copy()
    rolling_linear_residue = in_sample_residue.copy()

    true = diff_log_data.copy()[currency]
    for end_index in range(len(p_d_q_select_data), len(true)):
        used_data = true[end_index - len(p_d_q_select_data) : end_index]
        model = ARIMA(used_data, order=order)
        model_fit = model.fit()
        rolling_linear_prediction = rolling_linear_prediction.append(model_fit.forecast())
        rolling_linear_residue = rolling_linear_residue.append(true[end_index] - model_fit.forecast())
    
    return true, rolling_linear_prediction, rolling_linear_residue

In [38]:
p_list = [0, 1, 2, 3, 4, 5, 10, 20, 60]
q_list = [0, 1, 2, 3, 4, 5, 10, 20, 60]

## cad

In [39]:
p_d_q_select_cad = p_d_q_select_data.iloc[:,0]
p_d_q_select_cad

1981-01-02   -0.000239
1981-01-05    0.005124
1981-01-06    0.000357
1981-01-07    0.000238
1981-01-08    0.000000
                ...   
2016-12-26    0.002164
2016-12-27   -0.004875
2016-12-28    0.001221
2016-12-29    0.004195
2016-12-30    0.004715
Name: CADUSD Curncy, Length: 9391, dtype: float64

In [40]:
_1, p_value, _2, _3, _4, _5 = adfuller(p_d_q_select_cad, autolag='BIC')
print(p_value)

0.0


In [43]:
d = 0 # p_value < 5%

In [44]:
min_bic_order = grid_search_p_q(p_d_q_select_cad, p_list, d, q_list)

min_bic_order: (0, 0, 0)
min_bic: -75063.4642515508
order: (0, 0, 1) doesn't make bic smaller
bic: -75061.48100593289
order: (0, 0, 2) doesn't make bic smaller
bic: -75053.1778603082
order: (0, 0, 3) doesn't make bic smaller
bic: -75044.05979227813
order: (0, 0, 4) doesn't make bic smaller
bic: -75037.62317456333
order: (0, 0, 5) doesn't make bic smaller
bic: -75041.65480848728
order: (0, 0, 10) doesn't make bic smaller
bic: -75025.25204497507
order: (0, 0, 20) doesn't make bic smaller
bic: -74964.62147116702
order: (0, 0, 60) doesn't make bic smaller
bic: -74707.90064202817
order: (1, 0, 0) doesn't make bic smaller
bic: -75061.61391400915
order: (1, 0, 1) doesn't make bic smaller
bic: -75053.47821803983
order: (1, 0, 2) doesn't make bic smaller
bic: -75036.03817634698
order: (1, 0, 3) doesn't make bic smaller
bic: -75034.39736868483
order: (1, 0, 4) doesn't make bic smaller
bic: -75038.5603883536
order: (1, 0, 5) doesn't make bic smaller
bic: -75032.38608657129
order: (1, 0, 10) doesn

(0, 0, 0)

In [80]:
# min_bic_order = (0, 0, 0)

### recursive window

In [81]:
cad_true, cad_recursive_linear_prediction, cad_recursive_linear_residue = recursive_true_linear_residue(p_d_q_select_cad, min_bic_order, diff_log_data, 'CADUSD Curncy')

In [82]:
cad_true

1981-01-02   -0.000239
1981-01-05    0.005124
1981-01-06    0.000357
1981-01-07    0.000238
1981-01-08    0.000000
                ...   
2020-12-25   -0.000129
2020-12-28    0.001414
2020-12-29    0.002182
2020-12-30    0.005242
2020-12-31    0.001402
Name: CADUSD Curncy, Length: 10435, dtype: float64

In [83]:
cad_recursive_linear_prediction

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000012
2020-12-28   -0.000012
2020-12-29   -0.000012
2020-12-30   -0.000012
2020-12-31   -0.000011
Freq: B, Length: 10435, dtype: float64

In [84]:
cad_recursive_linear_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000117
2020-12-28    0.001426
2020-12-29    0.002194
2020-12-30    0.005254
2020-12-31    0.001413
Length: 10435, dtype: float64

### rolling window

In [85]:
cad_true, cad_rolling_linear_prediction, cad_rolling_linear_residue = rolling_true_linear_residue(p_d_q_select_cad, min_bic_order, diff_log_data, 'CADUSD Curncy')

In [86]:
cad_true

1981-01-02   -0.000239
1981-01-05    0.005124
1981-01-06    0.000357
1981-01-07    0.000238
1981-01-08    0.000000
                ...   
2020-12-25   -0.000129
2020-12-28    0.001414
2020-12-29    0.002182
2020-12-30    0.005242
2020-12-31    0.001402
Name: CADUSD Curncy, Length: 10435, dtype: float64

In [87]:
cad_rolling_linear_prediction

1981-01-02   -0.000018
1981-01-05   -0.000018
1981-01-06   -0.000018
1981-01-07   -0.000018
1981-01-08   -0.000018
                ...   
2020-12-25   -0.000002
2020-12-28   -0.000002
2020-12-29   -0.000002
2020-12-30   -0.000002
2020-12-31   -0.000001
Freq: B, Length: 10435, dtype: float64

In [88]:
cad_rolling_linear_residue

1981-01-02   -0.000221
1981-01-05    0.005142
1981-01-06    0.000374
1981-01-07    0.000255
1981-01-08    0.000018
                ...   
2020-12-25   -0.000126
2020-12-28    0.001416
2020-12-29    0.002184
2020-12-30    0.005244
2020-12-31    0.001403
Length: 10435, dtype: float64

In [45]:
cad_predict = diff_log_data.copy().iloc[:,0]
cad_predict.values[:] = 0
cad_resid = diff_log_data.copy().iloc[:,0]

## aud

In [46]:
p_d_q_select_aud = p_d_q_select_data.iloc[:,1]
p_d_q_select_aud

1981-01-02   -0.001016
1981-01-05    0.003806
1981-01-06    0.003623
1981-01-07    0.000000
1981-01-08   -0.002105
                ...   
2016-12-26    0.002227
2016-12-27   -0.000974
2016-12-28   -0.001114
2016-12-29    0.005835
2016-12-30   -0.001525
Name: AUDUSD Curncy, Length: 9391, dtype: float64

In [47]:
_1, p_value, _2, _3, _4, _5 = adfuller(p_d_q_select_aud, autolag='BIC')
print(p_value)

0.0


In [48]:
d = 0 # p_value < 5%

In [49]:
min_bic_order = grid_search_p_q(p_d_q_select_aud, p_list, d, q_list)

min_bic_order: (0, 0, 0)
min_bic: -65942.34037527566
order: (0, 0, 1) doesn't make bic smaller
bic: -65939.89207053914
order: (0, 0, 2) doesn't make bic smaller
bic: -65931.881700208
order: (0, 0, 3) doesn't make bic smaller
bic: -65926.12723605333
order: (0, 0, 4) doesn't make bic smaller
bic: -65919.63493552755
order: (0, 0, 5) doesn't make bic smaller
bic: -65911.1771524505
order: (0, 0, 10) doesn't make bic smaller
bic: -65868.96103871781
order: (0, 0, 20) doesn't make bic smaller
bic: -65802.29855802434
order: (0, 0, 60) doesn't make bic smaller
bic: -65489.40608211919
order: (1, 0, 0) doesn't make bic smaller
bic: -65939.74447577412
order: (1, 0, 1) doesn't make bic smaller
bic: -65932.16677623239
order: (1, 0, 2) doesn't make bic smaller
bic: -65920.71829712941
order: (1, 0, 3) doesn't make bic smaller
bic: -65919.01014748907
order: (1, 0, 4) doesn't make bic smaller
bic: -65910.8793186062
order: (1, 0, 5) doesn't make bic smaller
bic: -65902.03678736169
order: (1, 0, 10) doesn'

(0, 0, 0)

In [89]:
# min_bic_order = (0, 0, 0)

### recursive window

In [90]:
aud_true, aud_recursive_linear_prediction, aud_recursive_linear_residue = recursive_true_linear_residue(p_d_q_select_aud, min_bic_order, diff_log_data, 'AUDUSD Curncy')

In [91]:
aud_true

1981-01-02   -0.001016
1981-01-05    0.003806
1981-01-06    0.003623
1981-01-07    0.000000
1981-01-08   -0.002105
                ...   
2020-12-25    0.001052
2020-12-28   -0.003425
2020-12-29    0.003556
2020-12-30    0.010463
2020-12-31    0.001040
Name: AUDUSD Curncy, Length: 10435, dtype: float64

In [92]:
aud_recursive_linear_prediction

1981-01-02   -0.000058
1981-01-05   -0.000058
1981-01-06   -0.000058
1981-01-07   -0.000058
1981-01-08   -0.000058
                ...   
2020-12-25   -0.000047
2020-12-28   -0.000047
2020-12-29   -0.000048
2020-12-30   -0.000047
2020-12-31   -0.000046
Freq: B, Length: 10435, dtype: float64

In [93]:
aud_recursive_linear_residue

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2020-12-25    0.001100
2020-12-28   -0.003377
2020-12-29    0.003604
2020-12-30    0.010510
2020-12-31    0.001087
Length: 10435, dtype: float64

### rolling window

In [94]:
aud_true, aud_rolling_linear_prediction, aud_rolling_linear_residue = rolling_true_linear_residue(p_d_q_select_aud, min_bic_order, diff_log_data, 'AUDUSD Curncy')

In [95]:
aud_true

1981-01-02   -0.001016
1981-01-05    0.003806
1981-01-06    0.003623
1981-01-07    0.000000
1981-01-08   -0.002105
                ...   
2020-12-25    0.001052
2020-12-28   -0.003425
2020-12-29    0.003556
2020-12-30    0.010463
2020-12-31    0.001040
Name: AUDUSD Curncy, Length: 10435, dtype: float64

In [96]:
aud_rolling_linear_prediction

1981-01-02   -0.000058
1981-01-05   -0.000058
1981-01-06   -0.000058
1981-01-07   -0.000058
1981-01-08   -0.000058
                ...   
2020-12-25   -0.000015
2020-12-28   -0.000014
2020-12-29   -0.000014
2020-12-30   -0.000014
2020-12-31   -0.000013
Freq: B, Length: 10435, dtype: float64

In [97]:
aud_rolling_linear_residue

1981-01-02   -0.000959
1981-01-05    0.003863
1981-01-06    0.003681
1981-01-07    0.000058
1981-01-08   -0.002047
                ...   
2020-12-25    0.001068
2020-12-28   -0.003410
2020-12-29    0.003571
2020-12-30    0.010477
2020-12-31    0.001053
Length: 10435, dtype: float64

In [50]:
aud_predict = diff_log_data.copy().iloc[:,1]
aud_predict.values[:] = 0
aud_resid = diff_log_data.copy().iloc[:,1]

## gbp

In [ ]:
p_d_q_select_gbp = p_d_q_select_data['GBPUSD Curncy']
p_d_q_select_gbp

1981-01-02   -0.002934
1981-01-05    0.013966
1981-01-06    0.002894
1981-01-07   -0.004551
1981-01-08   -0.003323
                ...   
2016-12-26    0.000000
2016-12-27   -0.000896
2016-12-28   -0.003674
2016-12-29    0.002859
2016-12-30    0.006423
Name: GBPUSD Curncy, Length: 9391, dtype: float64

In [52]:
_1, p_value, _2, _3, _4, _5 = adfuller(p_d_q_select_gbp, autolag='BIC')
print(p_value)

0.0


In [53]:
d = 0 # p_value < 5%

In [54]:
min_bic_order = grid_search_p_q(p_d_q_select_gbp, p_list, d, q_list)

min_bic_order: (0, 0, 0)
min_bic: -68536.26970613962
min_bic_order: (0, 0, 1)
min_bic: -68537.66858170732
order: (0, 0, 2) doesn't make bic smaller
bic: -68528.60715367614
order: (0, 0, 3) doesn't make bic smaller
bic: -68520.32147711997
order: (0, 0, 4) doesn't make bic smaller
bic: -68511.38754931251
order: (0, 0, 5) doesn't make bic smaller
bic: -68502.78687593
order: (0, 0, 10) doesn't make bic smaller
bic: -68459.22018785166
order: (0, 0, 20) doesn't make bic smaller
bic: -68389.29487447883
order: (0, 0, 60) doesn't make bic smaller
bic: -68056.45883222729
min_bic_order: (1, 0, 0)
min_bic: -68537.71380950132
order: (1, 0, 1) doesn't make bic smaller
bic: -68528.57496592814
order: (1, 0, 2) doesn't make bic smaller
bic: -68519.51525467078
order: (1, 0, 3) doesn't make bic smaller
bic: -68510.8187103863
order: (1, 0, 4) doesn't make bic smaller
bic: -68502.9180285212
order: (1, 0, 5) doesn't make bic smaller
bic: -68493.3199722996
order: (1, 0, 10) doesn't make bic smaller
bic: -684

(1, 0, 0)

In [ ]:
# min_bic_order = (1, 0, 0)

### recursive window

In [72]:
gbp_true, gbp_recursive_linear_prediction, gbp_recursive_linear_residue = recursive_true_linear_residue(p_d_q_select_gbp, min_bic_order, diff_log_data, 'GBPUSD Curncy')

In [73]:
gbp_true

1981-01-02   -0.002934
1981-01-05    0.013966
1981-01-06    0.002894
1981-01-07   -0.004551
1981-01-08   -0.003323
                ...   
2020-12-25    0.000148
2020-12-28   -0.007996
2020-12-29    0.003784
2020-12-30    0.008994
2020-12-31    0.003297
Name: GBPUSD Curncy, Length: 10435, dtype: float64

In [74]:
gbp_recursive_linear_prediction

1981-01-02   -0.000071
1981-01-05   -0.000167
1981-01-06    0.000401
1981-01-07    0.000029
1981-01-08   -0.000221
                ...   
2020-12-25    0.000109
2020-12-28   -0.000048
2020-12-29   -0.000323
2020-12-30    0.000074
2020-12-31    0.000251
Freq: B, Length: 10435, dtype: float64

In [75]:
gbp_recursive_linear_residue

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2020-12-25    0.000038
2020-12-28   -0.007949
2020-12-29    0.004107
2020-12-30    0.008920
2020-12-31    0.003047
Length: 10435, dtype: float64

### rolling window

In [76]:
gbp_true, gbp_rolling_linear_prediction, gbp_rolling_linear_residue = rolling_true_linear_residue(p_d_q_select_gbp, min_bic_order, diff_log_data, 'GBPUSD Curncy')

In [77]:
gbp_true

1981-01-02   -0.002934
1981-01-05    0.013966
1981-01-06    0.002894
1981-01-07   -0.004551
1981-01-08   -0.003323
                ...   
2020-12-25    0.000148
2020-12-28   -0.007996
2020-12-29    0.003784
2020-12-30    0.008994
2020-12-31    0.003297
Name: GBPUSD Curncy, Length: 10435, dtype: float64

In [78]:
gbp_rolling_linear_prediction

1981-01-02   -0.000071
1981-01-05   -0.000167
1981-01-06    0.000401
1981-01-07    0.000029
1981-01-08   -0.000221
                ...   
2020-12-25    0.000171
2020-12-28    0.000020
2020-12-29   -0.000245
2020-12-30    0.000138
2020-12-31    0.000309
Freq: B, Length: 10435, dtype: float64

In [79]:
gbp_rolling_linear_residue

1981-01-02   -0.002864
1981-01-05    0.014132
1981-01-06    0.002493
1981-01-07   -0.004580
1981-01-08   -0.003102
                ...   
2020-12-25   -0.000024
2020-12-28   -0.008017
2020-12-29    0.004029
2020-12-30    0.008856
2020-12-31    0.002988
Length: 10435, dtype: float64

# Recursive linear_prediction and linear_residue

In [110]:
recursive_linear_prediction = pd.concat([cad_recursive_linear_prediction, aud_recursive_linear_prediction, gbp_recursive_linear_prediction], axis=1)
recursive_linear_prediction.columns = ['cad_recursive_linear_prediction', 'aud_recursive_linear_prediction', 'gbp_recursive_linear_prediction']
recursive_linear_prediction

,cad_recursive_linear_prediction,aud_recursive_linear_prediction,gbp_recursive_linear_prediction
1981-01-02,-0.000018,-0.000058,-0.000071
1981-01-05,-0.000018,-0.000058,-0.000167
1981-01-06,-0.000018,-0.000058,0.000401
1981-01-07,-0.000018,-0.000058,0.000029
1981-01-08,-0.000018,-0.000058,-0.000221
...,...,...,...
2020-12-25,-0.000012,-0.000047,0.000109
2020-12-28,-0.000012,-0.000047,-0.000048
2020-12-29,-0.000012,-0.000048,-0.000323
2020-12-30,-0.000012,-0.000047,0.000074


In [111]:
recursive_linear_prediction.to_csv('./recursive_linear_prediction.csv')

In [109]:
recursive_linear_residue = pd.concat([cad_recursive_linear_residue, aud_recursive_linear_residue, gbp_recursive_linear_residue], axis=1)
recursive_linear_residue.columns = ['cad_recursive_linear_residue', 'aud_recursive_linear_residue', 'gbp_recursive_linear_residue']
recursive_linear_residue

,cad_recursive_linear_residue,aud_recursive_linear_residue,gbp_recursive_linear_residue
1981-01-02,-0.000221,-0.000959,-0.002864
1981-01-05,0.005142,0.003863,0.014132
1981-01-06,0.000374,0.003681,0.002493
1981-01-07,0.000255,0.000058,-0.004580
1981-01-08,0.000018,-0.002047,-0.003102
...,...,...,...
2020-12-25,-0.000117,0.001100,0.000038
2020-12-28,0.001426,-0.003377,-0.007949
2020-12-29,0.002194,0.003604,0.004107
2020-12-30,0.005254,0.010510,0.008920


In [112]:
recursive_linear_residue.to_csv('./recursive_linear_residue.csv')

# Rolling linear_prediction and linear_residue

In [108]:
rolling_linear_prediction = pd.concat([cad_rolling_linear_prediction, aud_rolling_linear_prediction, gbp_rolling_linear_prediction], axis=1)
rolling_linear_prediction.columns = ['cad_rolling_linear_prediction', 'aud_rolling_linear_prediction', 'gbp_rolling_linear_prediction']
rolling_linear_prediction

,cad_rolling_linear_prediction,aud_rolling_linear_prediction,gbp_rolling_linear_prediction
1981-01-02,-0.000018,-0.000058,-0.000071
1981-01-05,-0.000018,-0.000058,-0.000167
1981-01-06,-0.000018,-0.000058,0.000401
1981-01-07,-0.000018,-0.000058,0.000029
1981-01-08,-0.000018,-0.000058,-0.000221
...,...,...,...
2020-12-25,-0.000002,-0.000015,0.000171
2020-12-28,-0.000002,-0.000014,0.000020
2020-12-29,-0.000002,-0.000014,-0.000245
2020-12-30,-0.000002,-0.000014,0.000138


In [113]:
rolling_linear_prediction.to_csv('./rolling_linear_prediction.csv')

In [107]:
rolling_linear_residue = pd.concat([cad_rolling_linear_residue, aud_rolling_linear_residue, gbp_rolling_linear_residue], axis=1)
rolling_linear_residue.columns = ['cad_rolling_linear_residue', 'aud_rolling_linear_residue', 'gbp_rolling_linear_residue']
rolling_linear_residue

,cad_rolling_linear_residue,aud_rolling_linear_residue,gbp_rolling_linear_residue
1981-01-02,-0.000221,-0.000959,-0.002864
1981-01-05,0.005142,0.003863,0.014132
1981-01-06,0.000374,0.003681,0.002493
1981-01-07,0.000255,0.000058,-0.004580
1981-01-08,0.000018,-0.002047,-0.003102
...,...,...,...
2020-12-25,-0.000126,0.001068,-0.000024
2020-12-28,0.001416,-0.003410,-0.008017
2020-12-29,0.002184,0.003571,0.004029
2020-12-30,0.005244,0.010477,0.008856


In [114]:
rolling_linear_residue.to_csv('./rolling_linear_residue.csv')